# Demo of InfluenceMapper

In [1]:
import json

from openai import OpenAI

import influencemapper
from influencemapper.author_org.infer_iarc import build_prompt as author_org_build_prompt
from influencemapper.study_org.infer_iarc import build_prompt as study_org_build_prompt
from influencemapper.author_org.infer_iarc import infer as author_org_infer
from influencemapper.study_org.infer_iarc import infer as study_org_infer
from influencemapper.util import infer_is_funded, collapse_relationship

### Setup the input
The following uses the disclosure text as the input to InfluenceMapper. For extracting the author-entity relationships from the text, authors information is required.   

In [2]:
disclosure_text = 'All the authors are employed by Cardno ChemRisk, a consulting firm that provides scientific advice to the government, corporations, law firms and various scientific/professional organizations. Cardno ChemRisk has been engaged by DePuy Orthopaedics, Inc., a manufacturer of prosthetic devices, some of which contain cobalt and chromium, to provide general consulting and expert advice on scientific matters, as well as litigation support. This paper was prepared and written exclusively by the authors, without review, or comment by DePuy employees or counsel. It is likely that this work will be relied upon in medical research, nutrition research and litigation. One of the authors (DJP) has previously testified on behalf of DePuy in hip implant litigation. It is possible that any or all of the authors may be called upon to serve as expert witnesses on behalf of DePuy. Funding for the preparation of this paper was provided by DePuy. The preparation of the paper, including conduct of the literature review, review of the individual papers, integration and synthesis of the findings, the conclusions drawn and recommendations made are the exclusive professional work product of the authors and may not necessarily be those of their employer or the financial sponsor of the review.'
authors = ['Brooke E Tvermoes',
 'Dennis J Paustenbach',
 'Brent D Kerger',
 'Brent L Finley',
 'Kenneth M Unice']

### Setup the OpenAI client 
Create a plain text file called `secret.txt` containing the OpenAI API key.

In [3]:
api_key = open('../secret_key').read().strip()
client = OpenAI(api_key=api_key)

## Author-Entity Relationship Extraction
### Build prompt, send it to OpenAI and get the result

In [4]:
prompt = author_org_build_prompt(authors, disclosure_text)
result = author_org_infer(client, prompt)

In [5]:
result

ParsedChatCompletion[Result](id='chatcmpl-AN9weCZbXLOpcMA61nBNoIjeVV2og', choices=[ParsedChoice[Result](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[Result](content='{"author_info":[{"author_name":"Brooke E Tvermoes","organization":[{"org_name":"Cardno ChemRisk","relationship_type":["Employee of"]},{"org_name":"DePuy Orthopaedics, Inc","relationship_type":["Consultant"]}]},{"author_name":"Dennis J Paustenbach","organization":[{"org_name":"Cardno ChemRisk","relationship_type":["Employee of"]},{"org_name":"DePuy Orthopaedics, Inc","relationship_type":["Consultant", "Expert testimony"]}]},{"author_name":"Brent D Kerger","organization":[{"org_name":"Cardno ChemRisk","relationship_type":["Employee of"]},{"org_name":"DePuy Orthopaedics, Inc","relationship_type":["Consultant"]}]},{"author_name":"Brent L Finley","organization":[{"org_name":"Cardno ChemRisk","relationship_type":["Employee of"]},{"org_name":"DePuy Orthopaedics, Inc","relationship_type":["Cons

Check if the prompt was succesfully processed

In [6]:
if result.choices[0].finish_reason != 'stop':
    print("Failure")

In [7]:
parsed_result = json.loads(result.choices[0].message.content)

The result would be a list of triplets stored in `parsed_result['author_info']`

In [8]:
an_author_info = parsed_result['author_info'][1]

Each author would be associated with multiple entities, and with each entity, the author would have various relationships.

In [9]:
# First entity associated
print(an_author_info['organization'][0]['org_name'])

Cardno ChemRisk


In [10]:
# Check if the entity is a funded one
print(infer_is_funded(an_author_info['organization'][0]['org_name']))

Likely


In [12]:
# Get the collapsed relationship
print(collapse_relationship(an_author_info['organization'][0]['relationship_type'][0]))

Direct financial relationship with entity


## Study-Entity Relationship Extraction
### Build prompt, send it to OpenAI and get the result

In [14]:
prompt2 = study_org_build_prompt(disclosure_text)
result2 = study_org_infer(client, prompt)

In [16]:
parsed_result = json.loads(result2.choices[0].message.content)

In [17]:
an_study_info = parsed_result['study_info'][1]

In [18]:
an_study_info

{'org_name': 'DePuy',
 'relationships': [{'relationship_type': 'Received research grant funds indirectly',
   'relationship_indication': 'Funding for the preparation of this paper was provided by DePuy'}]}